In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
url = "https://www.basketball-reference.com/leagues/NBA_2023_per_game.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
table = soup.find("table", {"id": "per_game_stats"})


headers = [ "Player", "Pos", "Age", "Tm", "G", "GS", "MP", "FG", "FGA", "FG%", "3P", "3PA", "3P%",
           "2P", "2PA", "2P%", "eFG%", "FT", "FTA", "FT%", "ORB", "DRB", "TRB", "AST", "STL", "BLK",
           "TOV", "PF", "PTS"]

# Extract the table rows
rows = []
for row in table.find_all("tr"):
    data = [cell.get_text() for cell in row.find_all("td")]
    if data:
        rows.append(data)

# Create a DataFrame from the extracted rows
df = pd.DataFrame(rows, columns=headers)

# Display the DataFrame
display(df)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,C,23,TOR,55,12,20.7,3.6,7.3,.485,...,.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
1,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,.597,...,.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,Bam Adebayo,C,25,MIA,75,75,34.6,8.0,14.9,.540,...,.806,2.5,6.7,9.2,3.2,1.2,0.8,2.5,2.8,20.4
3,Ochai Agbaji,SG,22,UTA,59,22,20.5,2.8,6.5,.427,...,.812,0.7,1.3,2.1,1.1,0.3,0.3,0.7,1.7,7.9
4,Santi Aldama,PF,22,MEM,77,20,21.8,3.2,6.8,.470,...,.750,1.1,3.7,4.8,1.3,0.6,0.6,0.8,1.9,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,.545,...,.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4
675,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,.429,...,.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2
676,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,.593,...,.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4
677,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,.627,...,.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5


In [2]:
df.to_csv('nba_2022_2023.csv', index=False)

In [3]:
# Find duplicate players
duplicates = df[df.duplicated(subset="Player", keep=False)]

# Filter the DataFrame to include only "TOT" rows for duplicate players
df_filtered = df[~((df["Player"].isin(duplicates["Player"])) & (df["Tm"] != "TOT"))]

# Specify the columns to convert to numeric
columns_to_convert = ['G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%',
                      'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

# Convert the specified columns to numeric
df_filtered[columns_to_convert] = df_filtered[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Display the filtered DataFrame
display(df_filtered)

/var/folders/st/39ct9mys46s6vz0ylf87j7540000gn/T/ipykernel_60441/3859767838.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[columns_to_convert] = df_filtered[columns_to_convert].apply(pd.to_numeric, errors='coerce')


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,C,23,TOR,55,12,20.7,3.6,7.3,0.485,...,0.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
1,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,0.597,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,Bam Adebayo,C,25,MIA,75,75,34.6,8.0,14.9,0.540,...,0.806,2.5,6.7,9.2,3.2,1.2,0.8,2.5,2.8,20.4
3,Ochai Agbaji,SG,22,UTA,59,22,20.5,2.8,6.5,0.427,...,0.812,0.7,1.3,2.1,1.1,0.3,0.3,0.7,1.7,7.9
4,Santi Aldama,PF,22,MEM,77,20,21.8,3.2,6.8,0.470,...,0.750,1.1,3.7,4.8,1.3,0.6,0.6,0.8,1.9,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,0.545,...,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4
675,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,0.429,...,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2
676,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,0.593,...,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4
677,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,0.627,...,0.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5
